In [16]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [17]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 04:11:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.80MB/s    in 0.2s    

2022-03-08 04:11:05 (4.80 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Games").getOrCreate()

In [35]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
# Load in tsv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Video_Games_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)
#df.show(10)

print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

df.show(10)


1785997
1785886
1785886
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Vide

In [36]:
df.dtypes
#string to int to date

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

For each notebook (one dataset per notebook), complete the following:

Count the number of records (rows) in the dataset.

Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

Load the DataFrames that correspond to tables into an RDS instance. Note: This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.

In [37]:
#Review #1
review_df = df.select(["review_id", "customer_id", "product_id","product_parent", "review_date"])
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-03-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-10-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-12-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-03-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-11-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-11-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-10-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-12-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-02-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-02-04|
|R105W3ID5NHQ48|   43562425|B002I0JNK8|     930811126| 2011-07-07|
|R106J0FROF1O34|   50248806|B0050SWUTQ|     861146594| 2014-03

In [38]:
from pyspark.sql.types import DateType
review_df = review_df.withColumn("review_date", review_df["review_date"].cast(DateType()))
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-03-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-10-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-12-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-03-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-11-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-11-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-10-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-12-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-02-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-02-04|
|R105W3ID5NHQ48|   43562425|B002I0JNK8|     930811126| 2011-07-07|
|R106J0FROF1O34|   50248806|B0050SWUTQ|     861146594| 2014-03

In [29]:
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [23]:
#Product #1
product_df = df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [23]:
#from pyspark.sql.types import IntegerType

In [30]:
#Customer #1
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [31]:
from pyspark.sql.types import IntegerType
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [25]:
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [32]:
#Customers #1
vine_df = df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [42]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com"
config = {"user":"postgres", "password":"AGautam4396", "driver" :"org.postgresql.Driver"}

In [47]:
review_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="review_id_table", mode="append", properties= config)

In [46]:
product_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="products", mode="append", properties= config)

In [ ]:
customers_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="customers", mode="append", properties= config)

In [ ]:
vine_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="vine_table", mode="append", properties= config)